In [ ]:
from analysis.paper import *
from boardlaw import sql, elos

In [ ]:
ags = sql.agent_query()

es = []
es.append(data.trial_elos(7))
es = pd.concat(es)
ags = ags.join(es, how='inner')

ags['test_flops'] = ags.test_nodes*(ags.train_flops/ags.samples)
ags['params'] = ags.width**2 * ags.depth

In [ ]:
(pn.ggplot(ags.query('test_nodes == 1 '), pn.aes(x='train_flops', y='ELO*elo', color='params', group='run'))
        + pn.geom_point(size=.25, show_legend=False)
        + pn.geom_line(size=.5, show_legend=False)
        + pn.scale_x_continuous(trans='log10')
        + pn.scale_color_cmap('viridis', trans='log10')
        + pn.labs(
            x='Test-time FLOPS',
            y='Elo v. perfect play')
        + plot.IEEE())

In [ ]:
boardsize = 7
ags = sql.agent_query().query('test_nodes == 64 & boardsize == 7')

trials = sql.trial_query(boardsize, 'bee/%')
trials = trials[trials.black_agent.isin(ags.index) & trials.white_agent.isin(ags.index)]
ws, gs = elos.symmetrize(trials)

ags['elo'] = elos.solve(ws, gs)

In [ ]:
mhx = sql.mohex_trial_query(boardsize)
black_wins = mhx[['black_agent', 'black_wins', 'white_wins']].dropna().set_index('black_agent')[['black_wins', 'white_wins']]
white_wins = mhx[['white_agent', 'black_wins', 'white_wins']].dropna().set_index('white_agent')[['black_wins', 'white_wins']]
black_wins = black_wins.groupby(black_wins.index).sum()
white_wins = white_wins.groupby(white_wins.index).sum()

rate = (black_wins.black_wins + white_wins.white_wins)/(black_wins.sum(1) + white_wins.sum(1))
rate.index = rate.index.astype(int)

mhx_elo = np.log(rate) - np.log(1 - rate)

In [ ]:
pd.concat({'old': ags.elo, 'new': mhx_elo}, 1).plot.scatter('old', 'new')

In [ ]:
mhx_elo.sort_values()